# Regions V2 - Living Income Benchmark V2 - CPI V2

Here we will map the li_benchmark_data.csv with countries.csv v2 and region.csv v1

In [1]:
import pandas as pd
import re

## Load files

In [2]:
countries = pd.read_csv("../countries_v2/countries.csv")

In [3]:
lib = pd.read_csv("./li_benchmark_data.csv")

In [4]:
lib = lib.dropna(how="all")

In [5]:
lib["id"] = lib.reset_index().index + 1

In [6]:
lib.columns

Index(['country', 'region', 'source', 'household_size', 'nr_adults', 'year',
       'LCU', 'USD', 'EUR', 'household_equiv', 'links', 'id'],
      dtype='object')

## FN

## Regions V2

In [7]:
def find_country(val):
    # manual match
    if val == "Turkey":
        val = "Turkiye"
    # EOL manual match
    find_country = countries[countries["country"].str.lower() == str(val).lower()]
    if find_country.empty:
        # Another custom way
        val_lower = str(val).lower()
        val_tmp = val_lower.lower().split(',')
        for index, row in countries.iterrows():
            # Convert the country name in the DataFrame to lowercase
            country_name = row['country'].lower()
            if val_lower == 'nan':
                return None
            if country_name not in val_tmp:
                continue
            return row["id"]
        # EOL Another custom way
        print("404", val)
        return val
    return find_country["id"].to_list()[0]

In [8]:
df = pd.read_excel("../lib_dataset.xlsx", sheet_name=0, header=4)

In [9]:
df = df.rename(columns={"Country": "country", "Region/area.1": "region"})

In [10]:
regions = df[["country", "region"]]

In [11]:
regions = regions[regions["region"].notna()]

In [12]:
regions['country'] = regions['country'].str.replace('\n', ' ').str.strip()
regions['region'] = regions['region'].str.replace('\n', ' ').str.strip()
regions['region'] = regions['region'].str.replace(r'^\s+|\s+?$', '', regex=True)

In [13]:
split_df = regions.assign(region=regions['region'].str.split(' / ')).explode('region')
split_df = regions.assign(region=regions['region'].str.split('/ ')).explode('region')
split_df = regions.assign(region=regions['region'].str.split(' /')).explode('region')
split_df = regions.assign(region=regions['region'].str.split('/')).explode('region')
split_df = split_df.drop_duplicates(subset=['country', 'region'])
split_df.reset_index(drop=True, inplace=True)

### Map regions with countries

In [14]:
split_df["country_id"] = split_df["country"].apply(find_country)

In [15]:
split_df.head()

,country,region,country_id
0,Argentina,Non-Metropolitan Urban,7
1,Argentina,All,7
2,Argentina,Rural,7
3,Bangladesh,Urban,15
4,Bangladesh,Satellite Cities,15


### Export Regions to CSV

In [16]:
group_df = split_df.groupby('region').agg({
    "country": "unique",
    "country_id": "unique"
}).reset_index()

In [17]:
group_df = group_df[group_df['region'].str.strip() != '']

In [18]:
group_df['region'] = group_df['region'].str.replace(r'^\s+|\s+?$', '', regex=True)

In [19]:
group_df['country_id'] = group_df['country_id'].apply(list)
group_df['country'] = group_df['country'].apply(list)

In [20]:
group_df["id"] = group_df.reset_index().index + 1

In [21]:
new_column_order = ["id", "region", "country_id", "country"]

In [22]:
group_df = group_df[new_column_order]

In [23]:
group_df.head()

,id,region,country_id,country
1,1,All,"[7, 19, 30, 33, 34, 38, 39, 46, 43, 56, 57, 67...","[Argentina, Belize, Burkina Faso, Cambodia, Ca..."
2,2,All coffee and cocoa growing regions,[145],[Peru]
3,3,Andhra Pradesh,[84],[India]
4,4,Baja California,[119],[Mexico]
5,5,Belém and the metropolitan area,[26],[Brazil]


In [24]:
regions = group_df

In [25]:
regions.to_csv('regions.csv', index=False)

## Living Income Benchmark V2

### Get country id

In [26]:
countries.columns

Index(['id', 'country', 'currency', 'abbreviation'], dtype='object')

In [27]:
def find_country(val):
    if val == "Turkey":
        val = "Turkiye"
    country = countries[countries["country"].str.lower() == str(val).lower()]
    if country.empty:
        print("404", val)
        return None
    return country["id"].to_list()[0]

In [28]:
lib["country_id"] = lib["country"].apply(find_country)

### Get region id

In [29]:
regions.columns

Index(['id', 'region', 'country_id', 'country'], dtype='object')

In [30]:
split_lib = lib.assign(region=lib['region'].str.split(' / ')).explode('region')
split_lib = lib.assign(region=lib['region'].str.split('/ ')).explode('region')
split_lib = lib.assign(region=lib['region'].str.split(' /')).explode('region')
split_lib = lib.assign(region=lib['region'].str.split('/')).explode('region')
split_lib = split_lib.drop_duplicates(subset=['country', 'region'])
split_lib.reset_index(drop=True, inplace=True)

In [31]:
lib = split_lib

In [32]:
lib['country'] = lib['country'].str.replace('\n', ' ').str.strip()
lib['region'] = lib['region'].str.replace('\n', ' ').str.strip()
lib['region'] = lib['region'].str.replace(r'^\s+|\s+?$', '', regex=True)

In [33]:
def find_region(val):
    region = regions[regions["region"].str.lower() == str(val).lower()]
    if region.empty:
        print("404", val, "*")
        return None
    return region["id"].to_list()[0]

In [34]:
lib["region_id"] = lib["region"].apply(find_region)

 Rondônia *
 Pará *theast
 Pará *thwest
 area *olitan the
 mineira *a Mata
404  *


In [39]:
lib = lib.dropna()

In [40]:
lib["id"] = lib.reset_index().index + 1

In [41]:
lib.columns

Index(['id', 'country_id', 'region_id', 'country', 'region', 'source',
       'household_size', 'nr_adults', 'year', 'LCU', 'USD', 'EUR',
       'household_equiv', 'links'],
      dtype='object')

In [42]:
lib = lib[['id', 'country_id',
       'region_id', 'country', 'region', 'source', 'household_size', 'nr_adults', 'year',
       'LCU', 'USD', 'EUR', 'household_equiv', 'links']]

In [43]:
lib.to_csv("./li_benchmark.csv", index=False)

## CPI V2